In [ ]:
!pip install cryptography

In [1]:
import boto3
from kmsgm import KMSManager
import botocore.exceptions
from cryptography.fernet import Fernet
import base64

s3 = boto3.client('s3')
kms_client = boto3.client('kms')
kms_manager = KMSManager(kms_client)

In [2]:
#kms_manager.create_key('alias/my-key')
kms_manager.select_key("68315a3a-60ce-4afd-a54b-523a90b6fc22")

Key 68315a3a-60ce-4afd-a54b-523a90b6fc22 selected.
Key 68315a3a-60ce-4afd-a54b-523a90b6fc22 is already Enabled.


In [3]:
alias_name = "alias/cloudtrail-s3-encryption-key"
kms_manager.create_alias(alias_name)

Couldn't create alias alias/cloudtrail-s3-encryption-key. An alias with the name arn:aws:kms:us-east-1:246591893798:alias/cloudtrail-s3-encryption-key already exists


In [11]:
kms_manager.describe_key()

Got key 68315a3a-60ce-4afd-a54b-523a90b6fc22:
{'AWSAccountId': '246591893798',
 'Arn': 'arn:aws:kms:us-east-1:246591893798:key/68315a3a-60ce-4afd-a54b-523a90b6fc22',
 'CreationDate': datetime.datetime(2024, 4, 3, 17, 54, 27, 576000, tzinfo=tzlocal()),
 'CustomerMasterKeySpec': 'SYMMETRIC_DEFAULT',
 'Description': '',
 'Enabled': True,
 'EncryptionAlgorithms': ['SYMMETRIC_DEFAULT'],
 'KeyId': '68315a3a-60ce-4afd-a54b-523a90b6fc22',
 'KeyManager': 'CUSTOMER',
 'KeySpec': 'SYMMETRIC_DEFAULT',
 'KeyState': 'Enabled',
 'KeyUsage': 'ENCRYPT_DECRYPT',
 'MultiRegion': False,
 'Origin': 'AWS_KMS'}


In [5]:
kms_manager.actual_state = "Enabled"

Key 68315a3a-60ce-4afd-a54b-523a90b6fc22 is already Enabled.


In [6]:
kms_manager.actual_state

'Enabled'

In [6]:
encrpyt_text = kms_manager.encrypt("Hello World")
print(encrpyt_text)

b'\x01\x02\x02\x00xM\xa3\xe9\xbf\x81\xc6Y\xda\x95U\x06\xe1\x1714\x80m\xac\x16\xa4\x9f\xa8\xd7\x8b\xeeOd\xcc\xe6\x1d\xcf\x87\x01zN\xf2\x85\xd0\xfc\xf3Qy\x1eL\x1e\xf6\xd8\xf8x\x00\x00\x00i0g\x06\t*\x86H\x86\xf7\r\x01\x07\x06\xa0Z0X\x02\x01\x000S\x06\t*\x86H\x86\xf7\r\x01\x07\x010\x1e\x06\t`\x86H\x01e\x03\x04\x01.0\x11\x04\x0cL\x1fp\xb3\xbdP\x91\xb5@4\x13\xcd\x02\x01\x10\x80&\xa0\xdf\x1ck\x91\x9c\x94\xed"3h\xaa\x90\xfdL\x05V?Ae\xc5\x11\xd8\xb4\xf9\x93\xa7+\xc4,\x18Ff\xaeP\x81I7'


In [7]:
decrypt_text = kms_manager.decrypt(encrpyt_text)
print(decrypt_text)	

Hello World


### Cifrar y descifrar un archivo
El programa de ejemplo utiliza claves AWS KMS para cifrar y descifrar un archivo.

Se crea una clave maestra, también denominada clave maestra de cliente o CMK, que se utiliza para generar una clave de datos. A continuación, la clave de datos se utiliza para cifrar un archivo de disco. La clave de datos cifrada se almacena dentro del archivo cifrado. Para descifrar el archivo, se descifra la clave de datos y luego se utiliza para descifrar el resto del archivo. Esta forma de utilizar las claves maestra y de datos se denomina cifrado envolvente.

Para cifrar y descifrar datos, el ejemplo utiliza el conocido paquete de criptografía de Python. Este paquete no forma parte de la biblioteca estándar de Python y debe instalarse por separado, por ejemplo, con el comando pip.

Las claves maestras se crean, administran y almacenan en AWS KMS. Una clave maestra de KMS también se denomina clave maestra de cliente o CMK. Se incurre en un costo de almacenamiento de AWS por cada CMK, por lo tanto, a menudo se utiliza una CMK para administrar varias claves de datos.

La función retrieve_cmk de ejemplo busca una CMK existente. Se especifica una descripción de clave cuando se crea una CMK, y esta descripción se utiliza para identificar y recuperar la clave deseada. Si existen muchas CMK, se procesan por lotes hasta que se encuentra la clave deseada o se examinan todas las claves.

Si la función de ejemplo encuentra la CMK deseada, devuelve tanto el ID de la CMK como su ARN (Amazon Resource Name). Cualquiera de estos identificadores se puede utilizar para hacer referencia a la CMK en llamadas posteriores a los métodos de AWS KMS.

In [104]:
def retrieve_cmk(description):
    """Retrieve an existing KMS CMK based on its description

    :param description: Description of CMK specified when the CMK was created
    :return Tuple(KeyId, KeyArn) where:
        KeyId: CMK ID
        KeyArn: Amazon Resource Name of CMK
    :return Tuple(None, None) if a CMK with the specified description was
    not found
    """

    # Retrieve a list of existing CMKs
    # If more than 100 keys exist, retrieve and process them in batches
    try:
        response = kms_client.list_keys()
    except ClientError as e:
        print(e)
        return None, None

    done = False
    while not done:
        for cmk in response['Keys']:
            # Get info about the key, including its description
            try:
                key_info = kms_client.describe_key(KeyId=cmk['KeyArn'])
            except ClientError as e:
                print(e)
                return None, None

            # Is this the key we're looking for?
            if key_info['KeyMetadata']['Description'] == description:
                return cmk['KeyId'], cmk['KeyArn']

        # Are there more keys to retrieve?
        if not response['Truncated']:
            # No, the CMK was not found
            print('A CMK with the specified description was not found')
            done = True
        else:
            # Yes, retrieve another batch
            try:
                response = kms_client.list_keys(Marker=response['NextMarker'])
            except ClientError as e:
                print(e)
                return None, None

    # All existing CMKs were checked and the desired key was not found
    return None, None

In [105]:
cmk_id, cmk_arn = retrieve_cmk("My Customer Master Key")
print(cmk_id, "|", cmk_arn)

68315a3a-60ce-4afd-a54b-523a90b6fc22 | arn:aws:kms:us-east-1:246591893798:key/68315a3a-60ce-4afd-a54b-523a90b6fc22


#### Create a data key
Para cifrar un archivo, la función de ejemplo create_data_key crea una clave de datos. La clave de datos es administrada por el cliente y no incurre en un costo de almacenamiento de AWS. El ejemplo crea una clave de datos para cada archivo que cifra, pero es posible utilizar una única clave de datos para cifrar varios archivos.

La función de ejemplo devuelve la clave de datos en forma de texto sin formato y cifrada. La forma de texto plano se utiliza para cifrar los datos. La forma cifrada se almacenará con el archivo cifrado. La clave de datos se asocia a una CMK capaz de descifrar la clave de datos cifrada cuando sea necesario.

In [102]:
def create_data_key(cmk_id, key_spec='AES_256'):
    """Generate a data key to use when encrypting and decrypting data

    :param cmk_id: KMS CMK ID or ARN under which to generate and encrypt the
    data key.
    :param key_spec: Length of the data encryption key. Supported values:
        'AES_128': Generate a 128-bit symmetric key
        'AES_256': Generate a 256-bit symmetric key
    :return Tuple(EncryptedDataKey, PlaintextDataKey) where:
        EncryptedDataKey: Encrypted CiphertextBlob data key as binary string
        PlaintextDataKey: Plaintext base64-encoded data key as binary string
    :return Tuple(None, None) if error
    """

    try:
        response = kms_client.generate_data_key(KeyId=cmk_id, KeySpec=key_spec)
    except ClientError as e:
        print(e)
        return None, None

    # Return the encrypted and plaintext data key
    return response['CiphertextBlob'], base64.b64encode(response['Plaintext'])

In [103]:
def encrypt_file(filename, cmk_id):
    """Encrypt a file using an AWS KMS CMK

    A data key is generated and associated with the CMK.
    The encrypted data key is saved with the encrypted file. This enables the
    file to be decrypted at any time in the future and by any program that
    has the credentials to decrypt the data key.
    The encrypted file is saved to <filename>.encrypted
    Limitation: The contents of filename must fit in memory.

    :param filename: File to encrypt
    :param cmk_id: AWS KMS CMK ID or ARN
    :return: True if file was encrypted. Otherwise, False.
    """
    NUM_BYTES_FOR_LEN = 4
    
    # Read the entire file into memory
    try:
        with open(filename, 'rb') as file:
            file_contents = file.read()
    except IOError as e:
        print(e)
        return False

    # Generate a data key associated with the CMK
    # The data key is used to encrypt the file. Each file can use its own
    # data key or data keys can be shared among files.
    # Specify either the CMK ID or ARN
    data_key_encrypted, data_key_plaintext = create_data_key(cmk_id)
    if data_key_encrypted is None:
        return False

    # Encrypt the file
    f = Fernet(data_key_plaintext)
    file_contents_encrypted = f.encrypt(file_contents)

    # Write the encrypted data key and encrypted file contents together
    try:
        with open(filename + '.encrypted', 'wb') as file_encrypted:
            file_encrypted.write(len(data_key_encrypted).to_bytes(NUM_BYTES_FOR_LEN,
                                                                  byteorder='big'))
            file_encrypted.write(data_key_encrypted)
            file_encrypted.write(file_contents_encrypted)
    except IOError as e:
        print(e)
        return False

    # For the highest security, the data_key_plaintext value should be wiped
    # from memory. Unfortunately, this is not possible in Python. However,
    # storing the value in a local variable makes it available for garbage
    # collection.
    return True

In [25]:
kms_arn = "arn:aws:kms:us-east-1:246591893798:key/68315a3a-60ce-4afd-a54b-523a90b6fc22"
if encrypt_file(file_name, kms_arn):
    print("File encrypted successfully")
else:
    print("File encryption failed")

Created new AWS KMS data key
File encrypted successfully


In [30]:
def decrypt_data_key(data_key_encrypted):
    """Decrypt an encrypted data key

    :param data_key_encrypted: Encrypted ciphertext data key.
    :return Plaintext base64-encoded binary data key as binary string
    :return None if error
    """

    # Decrypt the data key
    kms_client = boto3.client('kms')
    try:
        response = kms_client.decrypt(CiphertextBlob=data_key_encrypted)
    except ClientError as e:
        print(e)
        return None

    # Return plaintext base64-encoded binary data key
    return base64.b64encode((response['Plaintext']))

In [106]:
def decrypt_file(filename, original_name=False):
    """Decrypt a file encrypted by encrypt_file()

    The encrypted file is read from <filename>.encrypted
    The decrypted file is written to <filename>.decrypted

    :param filename: File to decrypt
    :return: True if file was decrypted. Otherwise, False.
    """
    NUM_BYTES_FOR_LEN = 4
    
    # Read the encrypted file into memory
    try:
        with open(filename, 'rb') as file:
            file_contents = file.read()
    except IOError as e:
        print(e)
        return False

    # The first NUM_BYTES_FOR_LEN bytes contain the integer length of the
    # encrypted data key.
    # Add NUM_BYTES_FOR_LEN to get index of end of encrypted data key/start
    # of encrypted data.
    data_key_encrypted_len = int.from_bytes(file_contents[:NUM_BYTES_FOR_LEN],
                                            byteorder='big') \
                             + NUM_BYTES_FOR_LEN
    data_key_encrypted = file_contents[NUM_BYTES_FOR_LEN:data_key_encrypted_len]

    # Decrypt the data key before using it
    data_key_plaintext = decrypt_data_key(data_key_encrypted)
    if data_key_plaintext is None:
        return False

    # Decrypt the rest of the file
    f = Fernet(data_key_plaintext)
    file_contents_decrypted = f.decrypt(file_contents[data_key_encrypted_len:])

    # Write the decrypted file contents
    try:
        if original_name:
            filename = original_name
        else:
            filename += '.decrypted'
        with open(filename, 'wb') as file_decrypted:
            file_decrypted.write(file_contents_decrypted)
    except IOError as e:
        print(e)
        return False

    # The same security issue described at the end of encrypt_file() exists
    # here, too, i.e., the wish to wipe the data_key_plaintext value from
    # memory.
    return filename

In [45]:
if decrypt_file("sample_file.pdf.encrypted", "sample_file_decrypted.pdf"):
    print("File decrypted successfully")
else:
    print("Failed to decrypt file")

File decrypted successfully


### Upload a file to S3

El cifrado predeterminado del bucket no anula la configuración de cifrado especificada en la carga de un nuevo objeto. Por ejemplo, si especifica el cifrado AES256 en su solicitud PutObject a un bucket con un cifrado SSE-KMS predeterminado, S3 respetará su solicitud y establecerá el cifrado SSE-S3. Le recomendamos que configure el cifrado predeterminado del bucket para imponer su esquema de cifrado en todas las cargas.

Para realizar esto es necesario añadir una declaración a la política del bucket que impondrá el tipo de cifrado. Si copias y pegas la plantilla de abajo, asegúrate de especificar el ACCOUNT_NUMBER en el documento de política JSON de abajo. Recuerde que aplicamos las políticas de cabecera de cifrado al contenido del bucket, no al bucket en sí mismo,-- incluya "/*" al final del ARN del bucket.

In [7]:
policy_s3 = {
    "Sid": "DenyIncorrectEncryptionHeader",
    "Effect": "Deny",
    "Principal": "*",
    "Action": "s3:PutObject",
    "Resource": "arn:aws:s3:::workshop-kms-s3-cloudtrail-<ACCOUNT_NUMBER>/*",
    "Condition": {
        "StringNotEquals": {
            "s3:x-amz-server-side-encryption": "aws:kms"
        }
    }
}

In [17]:
context = base64.b64encode(bytes('your_string', 'utf-8'))
context.hex()

'65573931636c397a64484a70626d633d'

In [22]:
file_name = "sample_file.pdf.encrypted"
bucket_name = "workshop-kms-s3-cloudtrail-246591893798"
objectkey = "sample_file_new.pdf.encrypted"

s3.upload_file(
    Filename=file_name, 
    Bucket=bucket_name, 
    Key=objectkey, 
    ExtraArgs={
        "ServerSideEncryption": "aws:kms", 
        "SSEKMSKeyId": cmk_id
    }
)